In [1]:
import pandas as pd
import numpy as np
import sklearn 
import json
import urllib
import requests
import json
import re
import matplotlib.pyplot as plt
%matplotlib inline
from imdbpie import Imdb


# imdb = Imdb()
# imdb = Imdb(anonymize=True) # to proxy requests

# Creating an instance with caching enabled
# Note that the cached responses expire every 2 hours or so.
# The API response itself dictates the expiry time)
imdb = Imdb(cache=True)

In [2]:
top_250_1 = imdb.top_250()
top_250_1 = pd.DataFrame(top_250_1)


In [ ]:
top_250_1.head()

,can_rate,image,num_votes,rating,tconst,title,type,year
0,True,{u'url': u'http://ia.media-imdb.com/images/M/M...,1679054,9.3,tt0111161,The Shawshank Redemption,feature,1994
1,True,{u'url': u'http://ia.media-imdb.com/images/M/M...,1148812,9.2,tt0068646,The Godfather,feature,1972
2,True,{u'url': u'http://ia.media-imdb.com/images/M/M...,785614,9.0,tt0071562,The Godfather: Part II,feature,1974
3,True,{u'url': u'http://ia.media-imdb.com/images/M/M...,1664202,9.0,tt0468569,The Dark Knight,feature,2008
4,True,{u'url': u'http://ia.media-imdb.com/images/M/M...,859341,8.9,tt0108052,Schindler's List,feature,1993


In [ ]:
from sklearn import preprocessing
top_250 = imdb.top_250()
top_250 = pd.DataFrame(top_250)

def review_find(imdb_id):
    try:
        return(imdb.get_title_reviews(imdb_id, max_results=30).strip("[<>]"))
    except:
        return('error')

def id_find(title):
    try:
        movie = imdb.search_for_title(title)
        return (movie[0]['imdb_id'])
    except:
        return('error')

def score_find(imdb_id):
    try:
        return(imdb.rating)
    except:
        return("error")
    
def cert_find(imdb_id):
    try:
        return(imdb.certification)
    except:
        return("error")

top_250['imdb_id'] = top_250['title'].apply(id_find)
top_250['reviews'] = top_250['imdb_id'].apply(review_find)
top_250['num_reviews'] = top_250['reviews'].apply(lambda x: len(x.split("Review:"))) #havent checked this

top_250['num_reviews_scaled'] = preprocessing.StandardScaler().fit_transform(top_250['num_reviews'])

top_250['rating_scaled'] = preprocessing.StandardScaler().fit_transform(top_250['rating'])


top_250.head()

In [ ]:
def certification_find(imdb_id):
    try:
        instance = imdb.get_title_by_id(imdb_id)
        return(instance.certification)
    except:
        return("error")

top_250['certification'] = top_250['imdb_id'].apply(certification_find)

cert_dums = pd.get_dummies(top_250['certification']
top_250 = pd.concat([top_250, cert_dums], axis = 1)

In [ ]:
##couldn't get reviews this way
# def review_find1(imdb_id):
#     try:
#         return(imdb.get_title_reviews(imdb_id, max_results=10))
#     except:
#         return('error')
    
# top_250['review_long'] = top_250['imdb_id'].apply(review_find1)

In [ ]:
#going to use omdb for reviews
# def plot_find(imdb_id):
#     try:
#         instance = imdb.get_title_by_id(imdb_id)
#         return(imdb.get_plot(instance, max_results=10))
#     except:
#         return('error')
    
# top_250['plot'] = top_250['imdb_id'].apply(plot_find)

In [ ]:
#reviews don't work here, going to use omdb api
# for i in top_250['reviews']:
#     for i2 in i:
#         print(i2)
    

In [ ]:
def credit_finder(imdb_id):
    try:
        title = imdb.get_title_by_id(imdb_id)
        return(title.credits)
    except:
        return("error")

top_250['credits'] = top_250['imdb_id'].apply(credit_finder)
top_250['num_listed_collabs'] = top_250['credits'].apply(lambda x: len(x.split("Person:"))                                                         
top_250['scaled_collab_count'] = preprocessing.StandardScaler().fit_transform(top_250['num_listed_collabs'])





In [ ]:
null_counts = top_250.count(axis=1)
null_counts_rows = top_250.count(axis=0)
null_counts.value_counts()

In [ ]:
null_counts_rows

In [ ]:
# #things to do:


# string parse plot
# count 'persons' in credits (DONE above)
# scale runtime (DONE here)
# convert year to date time
# convert released to datetime
# country to dummies
# scale num votes
# create dummys for type
# parse awards (somehow)
#create 'the' variable (DONE BELOW)
#need to make sure all data types are right

def intminutes(x):
    y = x.replace('min', '').strip()
    return int(y)

def integer_votes(x):
    y = x.replace(',', '').strip()
    return int(y)



top_250['num_votes'] = top_250['num_votes'].apply(integer_votes)
top_250['year'] = top_250['Year'].astype(int) #should i make this feature, years since release? 2016-x?
top_250['Runtime'] = top_250['Runtime'].apply(intminutes)
top_250['runtime_scaled'] = preprocessing.StandardScaler().fit_transform(df1['Runtime'])
top_250['num_votes_scaled'] = preprocessing.StandardScaler().fit_transform(top_250['num_votes'])
top_250['year_date'] = pd.to_datetime(top_250['Year'])# optional arguement for trying various options out should be on
top_250['release_date'] = pd.to_datetime(top_250['Release'])
top_250['the'] = top_250['title'].apply(lambda x: 1 if "the" in x else 0)
top_250['subtitle'] = top_250['title'].apply(lambda x: 1 if ":" in x else 0)

type_dums = pd.get_dummies(top_250['Type'])
top_250 = pd.concat([top_250, type_dums], axis = 1)
country_dums = pd.get_dummies(top_250['Country'])
top_250 = pd.concat([top_250, country_dums], axis = 1)
director_dums = pd.get_dummies(top_250['Director'])
top_250 = pd.concat([top_250, director_dums], axis = 1)
top_250['num_actors'] = top_250.apply(lambda x: len(top_250['Actors'].split(','))
genre_dums = pd.get_dummies(top_250['Genre'].split(", "))
top_250 = pd.concat([top_250, genre_dums], axis =1)
top_250['nominated'] = top_250.apply(lambda x: 1 if "nomin" in x else 0)
top_250['win'] = top_250.apply(lambda x: 1 if any(["win", 'won', 'winner', 'awarded', 'received'] in x else 0)





In [ ]:
top_250

In [ ]:
null_counts_rows.value_counts()

In [ ]:
top_250.describe()

In [ ]:
# for i in top_250['credits']:
#     print(i)
    
#need to split these persons w/ text parsing  
#also make feature for # of people involved in the movie


In [ ]:
top_250.columns = ['can_rate','image','num_votes','rating','tconst','Title','type','year','imdb_id','reviews','certification','review_long','credits'] #need to set title column as Title for merging
top_250.columns

In [ ]:
#function for all at once

ids = top_250['imdb_id']
#problem here is that its overwritting each row before, the last one comes back successfully
#tried again below with for loop outside of function
def info_adder(ids, dataframe):
    for i in ids:
        try:
            api_base_url = 'http://www.omdbapi.com/?i={}&plot=full&r=json'.format(i)
            api_response = requests.get(api_base_url)
            response = json.loads(api_response.text)
            new_info = pd.DataFrame(response.items())
            new_info.set_index(0, inplace=1)
            new_info = new_info.transpose()
            new_info.rename(columns={'imdbID':'imdb_id'}, inplace=True)
            dataframe = pd.merge(dataframe, new_info, how='left', on='imdb_id', inplace=1)
        except:
            continue
    return dataframe
            
top_250 = info_adder(ids, top_250)
top_250.head()



In [ ]:
#take 2 -- I think this should work

def info_adder2(imdb_id, dataframe):
    api_base_url = 'http://www.omdbapi.com/?i={}&plot=full&r=json'.format(imbd_id)
    api_response = requests.get(api_base_url)
    if api_response.status_code != 200:
        print(entry, api_response.status_code)
    try:
        response = json.loads(api_response.text)
        new_info = pd.DataFrame(response.items())
        new_info.set_index(0, inplace=1)
        new_info = new_info.transpose()
        new_info.rename(columns={'imdbID':'imdb_id'}, inplace=True)
        dataframe = pd.merge(dataframe, new_info, how='left', on='imdb_id', inplace=1)
        return(dataframe)
    except:
        return('error')


ids = top_250['imdb_id']
#problem here is that its overwritting each row before, the last one comes back successfully


for id_example in ids:
    top_250 = info_adder2(id_example, top_250)
top_250

In [ ]:
#function for all at once
# ids = top_250['imdb_id']


# # def info_adder(imdb_ids, dataframe):
# for i in imdb_ids:
# #         try:
#     api_base_url = 'http://www.omdbapi.com/?i={}&plot=full&r=json'.format(i)
#     print(api_base_url)
# #             api_response = requests.get(api_base_url)
#             response = json.loads(api_response.text)
#             new_info = pd.DataFrame(response.items())
#             new_info.set_index(0, inplace=1)
#             new_info = new_info.transpose()
#             new_info.rename(columns={'imdbID':'imdb_id'}, inplace=True)
#             dataframe = pd.merge(dataframe, new_info, how='left', on='imdb_id')
#         except:
#             continue
#     return(dataframe)
            
# top_250 = info_adder(ids, top_250)
# top_250.head()



In [ ]:
top_250

In [ ]:
###function for one at a time
# import string 


# def info_adder1(imdb_id):
#     try:
        
#         api_base_url = 'http://www.omdbapi.com/?i={}&plot=full&r=json'.format(imdb_id)
#         api_response = requests.get(api_base_url)
#         response = json.loads(api_response.text)
#         new_info = pd.DataFrame(response.items())
#         new_info.set_index(0, inplace=1)
#         new_info.rename(columns={'imdbID':'imdb_id'}, inplace=True)
#         new_info = new_info.transpose()
#     except:
#         return('error')
#     return(new_info)

# for i,v in enumerate(top_250['imdb_id']):
#      info_adder1(v)


In [ ]:
The Godfather

In [ ]:
top_250.head()

In [ ]:
api_base_url = "http://www.omdbapi.com/?t='Shawshank Redemption'&y=&plot=full&r=json"
api_response = requests.get(api_base_url)
response = json.loads(api_response.text)
shawshank = pd.DataFrame(response.items())
shawshank.set_index(0, inplace=1)
shawshank.transpose()
# = pd.DataFrame(response)
# shawshank.head()



In [ ]:
def get_gross(entry):
    response = requests.get('http://www.imdb.com/title/'+entry)
    html = response.text
    try:
        gross_list = re.findall("Gross:</h4>[ ]*\$([^ ]*)", html)
        gross = int(gross_list[0].replace(',', ''))
        print '.',
        return gross
    except Exception as ex:
        print
        print ex, entry, response.status_code
        return None
    
    
    
    
grosses = [(e, get_gross(e)) for e in ids]
df1 = pd.DataFrame(grosses, columns=['imdb_id', 'Gross'])

df1 = preprocessing.StandardScaler().fit_transform(df1['Gross'])
top_250 = pd.merge(top_250, df1, on="imdb_id", how='left')
top_250['gross_scaled'] = preprocessing.StandardScaler().fit_transform(top_250['Gross'])




In [ ]:
# string parse title and plot
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import PorterStemmer

cvec_plot = CountVectorizer(stop_words='english')
stemmer = PorterStemmer()


data_plot = cvec_plot.fit_transform(stemmer.stem(top_250['Plot'])).todense()
data_plot.columns()




# len(cvec.get_feature_names())
# X_train = pd.DataFrame(cvec.transform(data_train['data']).todense(),
#                        columns=cvec.get_feature_names())

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tv_ham = TfidfVectorizer(stop_words='english')
tv_ham.fit([ham])

tv_ham  = pd.DataFrame(tv_ham.transform([ham]).todense(),
             columns=tv_ham.get_feature_names())

df_ham_only = tv_ham.transpose().sort_values(0, ascending=False).head(15).transpose()

In [ ]:
cvec

cv = CountVectorizer(token_pattern=u'(?u)\\w+\.?\\w?\.? \\w+')
data = cv.fit_transform(df.Actors).todense()
columns = ['actor: '+c for c in cv.get_feature_names()]
actorsdf = pd.DataFrame(data, columns=columns)
actorsdf.head()
actorsdf.loc[0,actorsdf.iloc[0] != 0]
df = pd.concat([df, actorsdf], axis = 1)

In [ ]:
# title = imdb.get_title_by_id("tt1210166")
# for person in title.credits:
#     # check if they are a writer
#     if person.token == 'writers':
#         print(person.name + ' is a writer')
#     else:
#         print(person.name + ' is not a writer')

In [ ]:
# removing nulls still in df

top_250 = top_250.applymap(replace('N/A', np.nan))
top_250 = top_250.applymap(replace(None, np.nan))

top_250.info()

In [ ]:
# Goal Features:

# [‘rating_scaled’, ‘year’, ’num_reviews_scaled’, ‘rating_scaled’, ‘G’**??**, ‘PG’,  ‘PG-13’, ‘R’, ‘scaled_collab_count’, ’num_votes_scaled’, ‘release_date'
# ‘the’, ‘subtitle’, ‘num_actors’, ‘nominated’, ‘win’, ’gross_scaled’, ]

# look at: genre_dums, director_dums, country_dums, type_dums
# also look at any other features added with function info_adder2
#replace win with a better award feature: make list of awards that are valueable from the data (use text analysis)




In [ ]:
top_250['constant']=1
X = top_250.loc[:,['constant', 'year', 'num_reviews_scaled', 'G', 'PG','PG-13', 'R', 'scaled_collab_count', 'num_votes_scaled', 'release_date','the', 'subtitle', 'num_actors', 'nominated', 'win', 'gross_scaled’]]
#make sure to add dummy features
y = top_250.loc[:, 'rating'].values
X.describe()

In [ ]:
X.isnull().any() 

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split, cross_val_score
stratk = StratifiedKFold(y, n_folds=10, shuffle=True, random_state=66)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=50)


In [ ]:
dt = DecisionTreeClassifier(class_weight='balanced')
dt = DecisionTreeClassifier(class_weight='balanced')

dt.fit(X_train,y_train)
s_dt = cross_val_score(dt, X_scale, y, cv=stratk, n_jobs=-1, scoring='f1')



print("{} Score:\t{:0.3} ± {:0.3}".format("Decision Tree", s_dt.mean().round(3), s_dt_scale.std().round(3)))


In [ ]:
feature_importances_dt = pd.DataFrame(dt.feature_importances_,
                                   index = X.columns,
                                    columns=['importance']).sort_values('importance',
                                                                        ascending=False)
print(feature_importances_dt)

In [ ]:

rfc = RandomForestClassifier(class_weight='balanced', n_estimators=1000) #.948 before, .962 after

#X_scale = X_train     X_dums = Xd_train
rfc.fit(X_train,y_train)
s_rf = cross_val_score(rfc, X_scale, y, cv=stratk, n_jobs=-1, scoring='f1')

print("{} Score:\t{:0.3} ± {:0.3}".format("Random Forest Scaled", s_rf_scale.mean().round(3), s_rf_scale.std().round(3)))

In [ ]:
bag = BaggingClassifier(n_estimators=1000)

bag.fit(X_train,y_train)
s_bag = cross_val_score(bag, X_scale, y, cv=stratk, n_jobs=-1, scoring='f1')

print("{} Score:\t{:0.3} ± {:0.3}".format("Bagging Tree Scaled", s_bag.mean().round(3), s_bc_scale.std().round(3)))

In [ ]:
import matplotlib.pyplot as plt
from sklearn import metrics
def auc_plotting_function(rate1, rate2, rate1_name, rate2_name, curve_name):
    AUC = metrics.auc(rate1, rate2)
    # Plot of a ROC curve for class 1 (has_cancer)
    plt.figure(figsize=[11,9])
    plt.plot(rate1, rate2, label=curve_name + ' (area = %0.2f)' % AUC, linewidth=4)
    plt.plot([0, 1], [0, 1], 'k--', linewidth=4)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel(rate1_name, fontsize=18)
    plt.ylabel(rate2_name, fontsize=18)
    plt.title(curve_name + ' for salary > ~100,000', fontsize=18)
    plt.legend(loc="lower right")
    plt.show()

# plot receiving operator characteristic curve
def plot_roc(y_true, y_score):
    fpr, tpr, _ = metrics.roc_curve(y_true, y_score)
    auc_plotting_function(fpr, tpr, 'False Positive Rate', 'True Positive Rate', 'ROC')


plot_roc(y_test, ypred_bc_knn) ## Bagging KNN


In [ ]:
plot_roc(y_test, ypred_bc_scale) ## Bagging over trees with scaled features
